In [34]:
%%capture
%pip install scipy joblib

In [35]:
import numpy as np
import time
from scipy import stats
from joblib import Parallel, delayed
from scipy.stats import norm

In [36]:

def simulate_path_segment(m, r, T, sigma, S0, K, H, q, segment_n_paths, dt):
    dW = np.sqrt(dt) * np.random.randn(m, segment_n_paths)
    S = np.zeros((m + 1, segment_n_paths))
    S[0, :] = S0
    for i in range(1, m + 1):
        S[i, :] = S[i - 1, :] * np.exp((r - q - 0.5 * sigma ** 2) * dt + sigma * dW[i - 1, :])
        S[i, :] = np.where(S[i, :] < H, 0, S[i, :])  # Apply barrier condition

    payoffs = np.maximum(S[-1, :] - K, 0) * np.exp(-r * T)
    payoffs = np.where(S.any(axis=0), payoffs, 0)
    return payoffs

def price_down_and_out_call_brown(m, r, T, sigma, S0, K, H, q, n=10**7, n_jobs=8):
    n_paths = n
    n_steps = m
    dt = T / n_steps

    # Adjust the number of jobs and paths per job
    confidence_level=0.95
    paths_per_job = max(1, round(n_paths // n_jobs))  # Ensure at least one path per job

    # Adjust n_jobs if n_paths is smaller
    n_jobs = min(n_jobs, n_paths)

    # Running simulations in parallel
    results = Parallel(n_jobs=n_jobs)(delayed(simulate_path_segment)(
        m, r, T, sigma, S0, K, H, q, paths_per_job, dt) for _ in range(n_jobs))

    # Check if results are valid before concatenation
    if not results or any(len(res) == 0 for res in results):
        print("Error: No valid data returned from parallel tasks.")
        return 0, 0, 0

    # Concatenate results and calculate final values
    all_payoffs = np.concatenate(results)
    option_price = np.mean(all_payoffs)
    std_error = np.std(all_payoffs)
    sem = std_error / np.sqrt(n_paths)

    z_score = stats.norm.ppf(1 - (1 - confidence_level) / 2)
    confidence_interval = z_score * sem

    return option_price, sem, [round(option_price-confidence_interval,3), round(option_price+confidence_interval,3)]



In [37]:
def get_parameter_values(S0, K, T, r, q, sigma): 
    # d1 and d2 
    d1 = (np.log(S0/K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
       
    # Lambda value 
    lambda_ = (r - q + 0.5 * sigma**2) / sigma**2
    
    # Vanilla call value
    c = S0 * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2) 
    
    return lambda_, c

 
def down_and_call_book(S0, K, T, r, q, sigma,H, H_down, H_up):
    
    # Values for the different functions 
    lambda_, c = get_parameter_values(S0, K, T, r, q, sigma)
    
    # Value for the down and out
    y = np.log(H_down**2 / (S0*K)) / (sigma * np.sqrt(T)) + lambda_ * sigma * np.sqrt(T)
    
    # Values for the down and in 
    x1 = np.log(S0/H_up) / (sigma * np.sqrt(T)) + lambda_ * sigma * np.sqrt(T)
    y1 = np.log(H_up/S0) / (sigma * np.sqrt(T)) + lambda_ * sigma * np.sqrt(T)

    # Calculate option values for call
    if H <= K:
        #Down and out
        cdi = S0 * np.exp(-q * T) * (H_down/S0)**(2*lambda_) * norm.cdf(y) - K * np.exp(-r * T) * (H_down/S0)**(2*lambda_ - 2) * norm.cdf(y - sigma * np.sqrt(T))
        cdo = c  - cdi
        return cdo
    else:
        #Down and in
        cdo = S0 * np.exp(-q * T) * norm.cdf(x1) - K * np.exp(-r * T) * norm.cdf(x1 - sigma * np.sqrt(T)) 
        - S0 * np.exp(-q * T) * (H_up/S0)**(2*lambda_) * norm.cdf(y1) + K * np.exp(-r * T) * (H_up/S0)**(2*lambda_ - 2) * norm.cdf(y1 - sigma * np.sqrt(T))
        cdi = c - cdo
        
        return cdi


In [38]:
# Function to adjust the barrier for discrete monitoring
def adjusted_barrier(T, H, sigma,m, beta):
    # dT should be here, it "is the time between monitoring instants", p.325, also stated in book from michael at p.628
    delta_T = T / m

    # H_adj = H * np.exp( -1* beta * sigma * np.sqrt(T))
    H_adj_down = H * np.exp( -1* beta * sigma * np.sqrt(delta_T))
    H_adj_up = H * np.exp( beta * sigma * np.sqrt(delta_T))
    
    return H_adj_down, H_adj_up

In [39]:
################### Adjusted Barrier Matset thesis
### Barrier engineer method
def regression_beta_estimate(T, sigma, H, S0, m):
    beta_start = 0.5826
    Sigma_sqrt_T = sigma*np.sqrt(T/m)

    #### TODO find this as a function of 
    beta_end =  7.1239e-01 / (1 + np.exp(-3.3001e+01*(Sigma_sqrt_T + 5.9682e-02)))
       
    ### This is from the reg of increase point both Logistic and quadratic can be used
    H_log_start =  1.5542e-01 / (1 + np.exp(-2.9803e+01*(Sigma_sqrt_T - 6.4969e-02)))
    
    # Convert back to H value    
    H_start_increase = round(S0 * np.exp(-H_log_start))  # Determine the start of increase
    H_end = S0 - 1

    # Start with a constant beta
    beta = beta_start

    # If we've reached the start of the increase, switch to the polynomial curve
    if H >= H_start_increase:
        # These are placeholders and should be solved based on your specific conditions
        a = (beta_end - beta_start) / ((H_end - H_start_increase)**2)
        b = -2 * a * H_start_increase
        c = beta_start + a * H_start_increase**2
        
        # Polynomial growth formula
        beta = a * H**2 + b * H + c

    return beta

# Function to adjust the barrier for discrete monitoring
def adjusted_barrier_estimate(T, H, S0,sigma, m):
    
    # dT should be here, it "is the time between monitoring instants", p.325, also stated in book from michael at p.628
    delta_T = T / m
    beta = regression_beta_estimate(T, sigma, H, S0, m)

    ### adjust the beta value
    H_adj_down = H * np.exp(-1 * beta * sigma * np.sqrt(delta_T))
    H_adj_up = H * np.exp(beta * sigma * np.sqrt(delta_T))
    
    return H_adj_down, H_adj_up

In [40]:
def price_error(price_exact, other_price):
    return round(100*((price_exact - other_price)/price_exact),3 )
    

In [41]:
# Example usage:
S0 = 200  # Current stock price
K = 200   # Strike price
T = 4.2  # Time to maturity in years
r = 0.1   # Risk-free interest rate
q = 0.0   # Dividend yield
sigma = 0.4  # Volatility
H = 195  # Barrier
m = 50
n = 5*10**7
beta = 0.5826

### Barrier adjustment from Glasserman
H_adj_down, H_adj_up = adjusted_barrier(T, H, sigma,m, beta)
H_adj_down_est, H_adj_up_est = adjusted_barrier_estimate(T, H, S0,sigma, m)

# Calculate up-and-out call price
start = time.time()
price_exact, sem, confidence_interval = price_down_and_out_call_brown(m, r, T, sigma, S0, K, H, q,n)
price_continious = down_and_call_book(S0, K, T, r, q, sigma,H, H, H)
price_corrected = down_and_call_book(S0, K, T, r, q, sigma,H, H_adj_down, H_adj_up)
price_corrected_est = down_and_call_book(S0, K, T, r, q, sigma,H, H_adj_down_est, H_adj_up_est)

print("The process took, ", round(time.time()-start), " seconds") 
print(f"Price exact: {price_exact:.3f}, err: {sem:.4f}, interval: {confidence_interval}")
print(f"Price corrected: {price_corrected:.3f}, error, {price_error(price_exact, price_corrected)}")
print(f"Price thesis: {price_corrected_est:.3f}, error, {price_error(price_exact, price_corrected_est)}")
#print(f"Price continious: {price_continious:.3f}, error: {price_error(price_exact, price_continious)}")
#print(f"Sem, {sem:.4f}, Interval, {confidence_interval}")

The process took,  60  seconds
Price exact: 30.993, err: 0.0200, interval: [30.954, 31.032]
Price corrected: 28.347, error, 8.537
Price thesis: 30.947, error, 0.148
